In [ ]:
%reset

In [ ]:
import os

# Import packages needed to extract road network using a polygon
import osmnx as ox
import geopandas as gpd


# Import packages to extract buildings
import matplotlib.pyplot as plt
import osmnx as ox
from IPython.display import Image

# Other packages
import networkx as nx # to work with graphs
import pandas as pd
import numpy as np
import requests
import urllib
import urllib3
import scipy as sp # for the adjacency matrix

%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

In [ ]:
###### Extract networks using barrier polygons, extract USGS elevation for each node and save as graphml #####

# Function to keept trying when making elevations request
def make_remote_request(url: str, params: dict, encoding='utf-8'):
    """
    Makes the remote request
    Continues making attempts until it succeeds
    """

    count = 1
    while True:
        try:
            response = requests.get((url + urllib.parse.urlencode(params)))
        except (OSError, urllib3.exceptions.ProtocolError) as error:
            print('\n')
            print('*' * 20, 'Error Occured', '*' * 20)
            print(f'Number of tries: {count}')
            print(f'URL: {url}')
            print(error)
            print('\n')
            count += 1
            continue
        break

    return response

# Function to extract elevation using coordinates
def elevation_function(x):
    url = 'https://nationalmap.gov/epqs/pqs.php?'
    params = {'x': x[1],
              'y': x[0],
              'units': 'Meters',
              'output': 'json'}
    result = make_remote_request(url, params)
    return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

url = r'https://nationalmap.gov/epqs/pqs.php?' # USGS Elevation Point Query Service

# Loop within polygons folder and extract network for each barrier
rootdir = 'C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (os.path.join(subdir, file))
            file_path = os.path.join(subdir, file)
            try:
                # Read polygons
                poly = geopandas.read_file(file_path)
                # Extract just the geometry (shapely object) part
                poly_geo = poly['geometry'].iloc[0]
                # Clean it with a buffer
                poly_geo = poly_geo.buffer(0)
                
                # Pull the roads with OSMnx
                G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                print(len(G.nodes))
                # Give to each node a new index based on integers from 0 and then add the osmid as an attribute
                osmids = list(G.nodes)
                G = nx.relabel.convert_node_labels_to_integers(G)
                osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
                nx.set_node_attributes(G, osmid_values, 'osmid')
                
                # Create dataframe with lat and lon values to use with the elevation function
                Y = nx.get_node_attributes(G,'y')
                X= nx.get_node_attributes(G,'x')
                Ydf = pd.DataFrame.from_dict(Y, orient='index', columns=['lat'])
                Xdf = pd.DataFrame.from_dict(X, orient='index', columns=['lon'])
                df = pd.concat([Ydf, Xdf], axis=1)

                # Apply the function
                df['elevations'] = df.apply(elevation_function, axis=1)
                # Add elevation as a new attribute
                nx.set_node_attributes(G, df.elevations, 'Elevations')

                # Save street network as graph
                ox.save_graphml(G, filepath='C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Graph_Elevation\\{0}.graphml'.format(file))
                #nodes.to_file(driver = 'ESRI Shapefile', filename = 'C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Elevation\\{0}'.format(file))
                print("done")
            except:
                pass

In [ ]:
# Plot GCS behavior - individual barriers

            
G = nx.read_graphml(r"C:\Users\Sofia\Dropbox\Barriers_US\NorthCarolina\Graph_Elevation\NC22.graphml")
# Make a copy of the network to break
B = G

# Pull out elevation attribute
E = nx.get_node_attributes(G,'Elevations')
# Convert str values in float to be able to sort them 
E = dict(zip(E.keys(), [float(value) for value in E.values()]))
# Sort it based on elevation, min first
Sorted_E = sorted(E.items(), key=lambda item: item[1])
CCs = np.zeros([len(Sorted_E),2])
# Select first element of each tuple in the list (nodes ID):
FT = [i[0] for i in Sorted_E]
ST = [i[1] for i in Sorted_E]
for i in range(len(ST)):
    ST[i] = float(ST[i])

# Loop through all nodes
for i in range(0, len(FT)):
    # Find the node with lowest elevation from the list using i and remove it
    B.remove_nodes_from(FT[0:i])
    # Check number of connected components/clusters
    CCs[int(i),0] = nx.algorithms.components.number_weakly_connected_components(B)
    # Check giant component size
    CCs[int(i),1]= largest_cc = len(max(nx.weakly_connected_components(B), key=len))/len(FT)

# Plot
x_coord = 1 * np.arange(len(CCs))/len(CCs)
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(x_coord,CCs[:,0],':ok')
ax1.set_ylabel("Number connected components")
ax2.plot(x_coord,CCs[:,1],':s', color='#4682B4', label='Giant component size')
ax3 = ax2.twinx()
ax3.plot(x_coord,ST,':o', color='#9ACD32', label='Elevation')


ax2.set_ylabel("Giant Component Size",)
ax3.set_ylabel("Elevation")
ax2.set_xlabel("Fraction of removed nodes") 
ax1.set_title('NC22', fontsize=22)


f.legend(loc="lower center", bbox_to_anchor=(0.5, 0.45), ncol=2, frameon=False)

f.tight_layout()

# Calculate robustness following Schneider's equation (2011)

s= sum(CCs[:,1])
r= s/len(FT)
print(r)

# Add text with R value
plt.text( 0.75, 0.20, r'R=0.42',
    ha='left', va='top',
    transform=f.transFigure
)

# Plot
plt.show()
plt.rcParams["font.size"]= 20
plt.rcParams["figure.figsize"] = (15,15)


# Save figure
f.savefig("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Figures\\NC22.jpeg")

In [ ]:
# Iterator to plot connected components, GCS and elevation for all barriers in North Carolina

rootdir = 'C:\\Users\\Sofia\\Dropbox\\NETWORKS\\NorthCarolina\\Graph_Elevation'
extensions = ('.graphml')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (os.path.join(subdir, file))
            file_path = os.path.join(subdir, file)
            
            G = nx.read_graphml(file_path)
            # Make a copy of the network to break
            B = G

            # Pull out elevation attribute
            E = nx.get_node_attributes(G,'Elevations')
            # Convert str values in float to be able to sort them 
            E = dict(zip(E.keys(), [float(value) for value in E.values()]))
            # Sort it based on elevation, min first
            Sorted_E = sorted(E.items(), key=lambda item: item[1])
            CCs = np.zeros([len(Sorted_E),2])
            # Select first element of each tuple in the list (nodes ID):
            FT = [i[0] for i in Sorted_E]
            ST = [i[1] for i in Sorted_E]
            for i in range(len(ST)):
                ST[i] = float(ST[i])

            # Loop through all nodes
            for i in range(0, len(FT)):
                # Find the node with lowest elevation from the list using i and remove it
                B.remove_nodes_from(FT[0:i])
                # Check number of connected components/clusters
                CCs[int(i),0] = nx.algorithms.components.number_weakly_connected_components(B)
                # Check giant component size
                CCs[int(i),1]= largest_cc = len(max(nx.weakly_connected_components(B), key=len))/len(FT)

            # Plot
            x_coord = 1 * np.arange(len(CCs))/len(CCs)
            f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
            ax1.plot(x_coord,CCs[:,0],':ok')
            ax1.set_ylabel("Number connected components")
            ax2.plot(x_coord,CCs[:,1],':s', color='#4682B4', label='Giant component size')
            ax3 = ax2.twinx()
            ax3.plot(x_coord,ST,':o', color='#9ACD32', label='Elevation')


            ax2.set_ylabel("Giant Component Size",)
            ax3.set_ylabel("Elevation")
            ax2.set_xlabel("Fraction of removed nodes") 
            title= file.replace('.graphml', '')
            ax1.set_title(title, fontsize=22)


            f.legend(loc="lower center", bbox_to_anchor=(0.5, 0.45), ncol=2, frameon=False)
        
            f.tight_layout()

            # Calculate robustness following Schneider's equation (2011)

            s= sum(CCs[:,1])
            r= round(s/len(FT),2)
            

            # Add text with R value
            plt.text( 0.75, 0.20, r'R=%s'%(r),
                ha='left', va='top',
                transform=f.transFigure
            )

            # Plot
            plt.show()
            plt.rcParams["font.size"]= 20
            plt.rcParams["figure.figsize"] = (15,15)

            # Save figure
            f.savefig("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Figures\\{0}.jpeg".format(title))

In [ ]:
#### Calculate area of barrier polygons ####

# Loop within polygons folder and extract network for each barrier
rootdir = 'C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (os.path.join(subdir, file))
            file_path = os.path.join(subdir, file)
            
            # Read polygons
            poly = gpd.read_file(file_path)
            poly_prj = poly.copy()
            poly_prj = poly_prj.to_crs("EPSG:6542")
            poly_prj["Area_km2"] = poly_prj['geometry'].area/10**6
            print(poly_prj["Area_km2"])

In [ ]:
#### Calculate statistics ####

# Read polygons
poly = gpd.read_file("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Polygons\\NC22_geo.shp")
# Extract just the geometry (shapely object) part
poly_geo = poly['geometry'].iloc[0]
# Clean it with a buffer
poly_geo = poly_geo.buffer(0)
poly_geo.is_valid
# Project polygon and calculate area
poly_prj=ox.project_gdf(poly)
area=poly_prj.area
print(area/10**6)

# Pull network
G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)

# Calculate network area based on convex hull
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
print(graph_area_m/10**6)

# show some basic stats about the network
stats = ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist='euclidean')

# delete the no longer needed dict elements
del stats['streets_per_node_counts']
del stats['streets_per_node_proportion']

# load as a pandas dataframe
df = pd.DataFrame(pd.Series(stats, name='value'))
df.round(1)


In [ ]:
# Extended stats - I haven't run this yet, that's just the code to do it.
extended_stats = ox.extended_stats(G, ecc=True, bc=True, cc=True)
for key, value in extended_stats.items():
    stats[key] = value
pd.Series(stats)

In [ ]:

#### Extract building footprints and save them as shp ####

rootdir = 'C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            ext = os.path.splitext(file)[-1].lower()
            if ext in extensions:
                print (os.path.join(subdir, file))
                file_path = os.path.join(subdir, file)
            
                # import polygon
                polygon = gpd.read_file(file_path)     
                # extract just the geometry (shapely object) part
                geom = polygon['geometry'].iloc[0]         
                # make geometry valid
                geom = geom.buffer(0)
                print(geom.is_valid)
            
                # extract buildings using polygon and save footprints as shp
                footprints = ox.geometries_from_polygon(geom, tags={'building':True})
                # project footprints
                footprints = ox.project_gdf(footprints)
                # get indexes where name column has value 'node'
                points = footprints[footprints['element_type'] == 'node'].index
                # delete these row indexes from dataFrame
                footprints.drop(points , inplace=True)
                
                # calculate areas in square meters
                footprints["Area_m2"] = footprints.area
               
                # save footprints as shp
                footprints_save = footprints.applymap(lambda x: str(x) if isinstance(x, list) else x)
                footprints_save.drop(labels='nodes', axis=1).to_file('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Buildings2\\{0}'.format(file))
            
        except:
            pass


In [ ]:
#### Calculate number of buildings and total building area ####

rootdir = 'C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Buildings'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (os.path.join(subdir, file))
            file_path = os.path.join(subdir, file)

            footprints= gpd.read_file(file_path)
            print(len(footprints))
            total_area = sum(footprints["Area_m2"])
            print(total_area/10**6)
                
                

In [ ]:
#### Connectivity: Remove nodes until the network breaks ####
#This has the problem that once one single node is isolated, the whole network disconnects, even though there is still a giant component and the network is still viable

G = nx.read_graphml(r"C:\Users\Sofia\Dropbox\Barriers_US\NorthCarolina\Graph_Elevation\NC4_geo.shp.graphml")

#make a copy of the network to break
B = G


# pull out elevation attribute 
E = nx.get_node_attributes(B,'Elevations')
#sort it based on elevation, min first
Sorted_E = sorted(E.items(), key=lambda item: item[1])
#Start at first entry of the list
i = 0
#assume you start with a connected network
#loop through until the network breaks
while True:
    #find the node with lowest elevation from the list using i and remove it
    B.remove_node((Sorted_E[i][0]))
    print(i)
    #check for connection
    NotConnected = nx.algorithms.components.is_weakly_connected(B)
    if NotConnected == False:
        #Stop loop
        break
  #increment the index
    i+= 1
# print the node that broke the system
print(i)
print(Sorted_E[i])

In [ ]:
# Plot histogram to with elevations

G = nx.read_graphml(r"C:\Users\Sofia\Dropbox\Barriers_US\NorthCarolina\Graph_Elevation\NC4.graphml")

# Convert network nodes into dataframe for plotting
G2 = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
Elev= pd.to_numeric(G2.Elevations)
# Stair histogram for elevations

def find_bins(observations, width):
    minimmum = np.min(observations)
    maximmum = np.max(observations)
    bound_min = -1.0 * (minimmum % width - minimmum)
    bound_max = maximmum - maximmum % width + width
    n = int((bound_max - bound_min) / width) + 1
    bins = np.linspace(bound_min, bound_max, n)
    return bins

bins = find_bins(Elev, 0.5)

fig = plt.hist(Elev, histtype='step', bins=bins)

plt.title('NC4')
plt.xlabel("Elevation")
plt.ylabel("Frequency")
plt.rcParams["figure.figsize"] = (10,10)
plt.show()
